<a href="https://colab.research.google.com/github/PawaritL/BayesianLSTM/blob/master/Energy_Consumption_Predictions_with_Bayesian_LSTMs_in_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Energy Consumption Predictions with Bayesian LSTMs in PyTorch

Author: Pawarit Laosunthara



# **Important Note for GitHub Readers:**
Please click the **Open in Colab** button above in order to view all **interactive visualizations**.

This notebook demonstrates an implementation of an (Approximate) Bayesian Recurrent Neural Network in PyTorch, originally inspired by the *Deep and Confident Prediction for Time Series at Uber* (https://arxiv.org/pdf/1709.01907.pdf)

<br>

In this approach, Monte Carlo dropout is used to **approximate** Bayesian inference, allowing our predictions to have explicit uncertainties and confidence intervals. This property makes Bayesian Neural Networks highly appealing to critical applications requiring uncertainty quantification.
The *Appliances energy prediction* dataset used in this example is from the UCI Machine Learning Repository (https://archive.ics.uci.edu/ml/datasets/Appliances+energy+prediction)


**Note:** this notebook purely serves to demonstrate the implementation of Bayesian LSTMs (Long Short-Term Memory) networks in PyTorch. Therefore, extensive data exploration and feature engineering is not part of the scope of this investigation.

# Preliminary Data Wrangling

**Selected Columns:**

For simplicity and speed when running this notebook, only temporal and autoregressive features are used.

- date time year-month-day hour:minute:second, sampled every 10 minutes \
- Appliances, energy use in Wh for the corresponding 10-minute timestamp \
- day_of_week, where Monday corresponds to 0 \
- hour_of_day


In [ ]:
import pandas as pd

In [ ]:
energy_df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/00374/energydata_complete.csv')

energy_df['date'] = pd.to_datetime(energy_df['date'])

energy_df['month'] = energy_df['date'].dt.month.astype(int)
energy_df['day_of_month'] = energy_df['date'].dt.day.astype(int)

# day_of_week=0 corresponds to Monday
energy_df['day_of_week'] = energy_df['date'].dt.dayofweek.astype(int)
energy_df['hour_of_day'] = energy_df['date'].dt.hour.astype(int)

selected_columns = ['date', 'day_of_week', 'hour_of_day', 'Appliances']
energy_df = energy_df[selected_columns]

In [ ]:
energy_df.head()

,date,day_of_week,hour_of_day,Appliances
0,2016-01-11 17:00:00,0,17,60
1,2016-01-11 17:10:00,0,17,60
2,2016-01-11 17:20:00,0,17,50
3,2016-01-11 17:30:00,0,17,50
4,2016-01-11 17:40:00,0,17,60


## Time Series Transformations

1. The dataset is to be re-sampled at an hourly rate for more meaningful analytics.

2. To alleviate exponential effects, the target variable is log-transformed as per the Uber paper.

3. For simplicity and speed when running this notebook, only temporal and autoregressive features, namely `day_of_week`, `hour_of_day`, \
and previous values of `Appliances` are used as features

In [ ]:
import numpy as np

resample_df = energy_df.set_index('date').resample('1H').mean()
resample_df['date'] = resample_df.index
resample_df['log_energy_consumption'] = np.log(resample_df['Appliances'])

datetime_columns = ['date', 'day_of_week', 'hour_of_day']
target_column = 'log_energy_consumption'

feature_columns = datetime_columns + ['log_energy_consumption']

# For clarity in visualization and presentation, 
# only consider the first 150 hours of data.
resample_df = resample_df[feature_columns]

In [ ]:
import plotly.express as px

plot_length = 150
plot_df = resample_df.copy(deep=True).iloc[:plot_length]
plot_df['weekday'] = plot_df['date'].dt.day_name()

fig = px.line(plot_df,
              x="date",
              y="log_energy_consumption", 
              color="weekday", 
              title="Log of Appliance Energy Consumption vs Time")
fig.show()

# Prepare Training Data

For this example, we will use sliding windows of 10 points per each window (equivalent to 10 hours) to predict each next point. The window size can be altered via the `sequence_length` variable.

Min-Max scaling has also been fitted to the training data to aid the convergence of the neural network. 

In [ ]:
from sklearn.preprocessing import MinMaxScaler

def create_sliding_window(data, sequence_length, stride=1):
    X_list, y_list = [], []
    for i in range(len(data)):
      if (i + sequence_length) < len(data):
        X_list.append(data.iloc[i:i+sequence_length:stride, :].values)
        y_list.append(data.iloc[i+sequence_length, -1])
    return np.array(X_list), np.array(y_list)

train_split = 0.7
n_train = int(train_split * len(resample_df))
n_test = len(resample_df) - n_train

features = ['day_of_week', 'hour_of_day', 'log_energy_consumption']
feature_array = resample_df[features].values

# Fit Scaler only on Training features
feature_scaler = MinMaxScaler()
feature_scaler.fit(feature_array[:n_train])
# Fit Scaler only on Training target values
target_scaler = MinMaxScaler()
target_scaler.fit(feature_array[:n_train, -1].reshape(-1, 1))

# Transfom on both Training and Test data
scaled_array = pd.DataFrame(feature_scaler.transform(feature_array),
                            columns=features)

sequence_length = 10
X, y = create_sliding_window(scaled_array, 
                             sequence_length)

X_train = X[:n_train]
y_train = y[:n_train]

X_test = X[n_train:]
y_test = y[n_train:]

# Define Bayesian LSTM Architecture

To demonstrate a simple working example of the Bayesian LSTM, a model with a similar architecture and size to that in Uber's paper has been used a starting point. The network architecture is as follows:

Encoder-Decoder Stage:
 - A uni-directional LSTM with 2 stacked layers & 128 hidden units acting as an encoding layer to construct a fixed-dimension embedding state
 - A uni-directional LSTM with 2 stacked layers & 32 hidden units acting as a decoding layer to produce predictions at future steps
 - Dropout is applied at **both** training and inference for both LSTM layers


 Predictor Stage:
 - 1 fully-connected output layer with 1 output (for predicting the target value) to produce a single value for the target variable


By allowing dropout at both training and testing time, the model simulates random sampling, thus allowing varying predictions that can be used to estimate the underlying distribution of the target value, enabling explicit model uncertainties.


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

class BayesianLSTM(nn.Module):

    def __init__(self, n_features, output_length, batch_size):

        super(BayesianLSTM, self).__init__()

        self.batch_size = batch_size # user-defined

        self.hidden_size_1 = 128 # number of encoder cells (from paper)
        self.hidden_size_2 = 32 # number of decoder cells (from paper)
        self.stacked_layers = 2 # number of (stacked) LSTM layers for each stage
        self.dropout_probability = 0.5 # arbitrary value (the paper suggests that performance is generally stable across all ranges)

        self.lstm1 = nn.LSTM(n_features, 
                             self.hidden_size_1, 
                             num_layers=self.stacked_layers,
                             batch_first=True)
        self.lstm2 = nn.LSTM(self.hidden_size_1,
                             self.hidden_size_2,
                             num_layers=self.stacked_layers,
                             batch_first=True)
        
        self.fc = nn.Linear(self.hidden_size_2, output_length)
        self.loss_fn = nn.MSELoss()
        
    def forward(self, x):
        batch_size, seq_len, _ = x.size()

        hidden = self.init_hidden1(batch_size)
        output, _ = self.lstm1(x, hidden)
        output = F.dropout(output, p=self.dropout_probability, training=True)
        state = self.init_hidden2(batch_size)
        output, state = self.lstm2(output, state)
        output = F.dropout(output, p=self.dropout_probability, training=True)
        output = output[:, -1, :] # take the last decoder cell's outputs
        y_pred = self.fc(output)
        return y_pred
        
    def init_hidden1(self, batch_size):
        hidden_state = Variable(torch.zeros(self.stacked_layers, batch_size, self.hidden_size_1))
        cell_state = Variable(torch.zeros(self.stacked_layers, batch_size, self.hidden_size_1))
        return hidden_state, cell_state
    
    def init_hidden2(self, batch_size):
        hidden_state = Variable(torch.zeros(self.stacked_layers, batch_size, self.hidden_size_2))
        cell_state = Variable(torch.zeros(self.stacked_layers, batch_size, self.hidden_size_2))
        return hidden_state, cell_state
    
    def loss(self, pred, truth):
        return self.loss_fn(pred, truth)

    def predict(self, X):
        return self(torch.tensor(X, dtype=torch.float32)).view(-1).detach().numpy()

### Begin Training

To train the Bayesian LSTM, we use the ADAM optimizer along with mini-batch gradient descent (`batch_size = 128`). For quick demonstration purposes, the model is trained for 150 epochs.

The Bayesian LSTM is trained on the first 70% of data points, using the aforementioned sliding windows of size 10. The remaining 30% of the dataset is held out purely for testing.

In [ ]:
n_features = scaled_array.shape[-1]
sequence_length = 10
output_length = 1

batch_size = 128
n_epochs = 150
learning_rate = 0.01

bayesian_lstm = BayesianLSTM(n_features=n_features,
                             output_length=output_length,
                             batch_size = batch_size)

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(bayesian_lstm.parameters(), lr=learning_rate)

In [ ]:
bayesian_lstm.train()

for e in range(1, n_epochs+1):
    for b in range(0, len(X_train), batch_size):
        features = X_train[b:b+batch_size,:,:]
        target = y_train[b:b+batch_size]    

        X_batch = torch.tensor(features,dtype=torch.float32)    
        y_batch = torch.tensor(target,dtype=torch.float32)

        output = bayesian_lstm(X_batch)
        loss = criterion(output.view(-1), y_batch)  

        loss.backward()
        optimizer.step()        
        optimizer.zero_grad() 

    if e % 10 == 0:
      print('epoch', e, 'loss: ', loss.item())

epoch 10 loss:  0.021331658586859703
epoch 20 loss:  0.01902003400027752
epoch 30 loss:  0.01965978927910328
epoch 40 loss:  0.017205342650413513
epoch 50 loss:  0.01645931601524353
epoch 60 loss:  0.017352448776364326
epoch 70 loss:  0.011554614640772343
epoch 80 loss:  0.013196350075304508
epoch 90 loss:  0.0104022566229105
epoch 100 loss:  0.008596537634730339
epoch 110 loss:  0.00793543178588152
epoch 120 loss:  0.006831762846559286
epoch 130 loss:  0.007397405803203583
epoch 140 loss:  0.0039006543811410666
epoch 150 loss:  0.004320652689784765


# Evaluating Model Performance

The Bayesian LSTM implemented is shown to produce reasonably accurate and sensible results on both the training and test sets, often comparable to other existing frequentist machine learning and deep learning methods.



In [ ]:
offset = sequence_length

def inverse_transform(y):
  return target_scaler.inverse_transform(y.reshape(-1, 1))

training_df = pd.DataFrame()
training_df['date'] = resample_df['date'].iloc[offset:n_train + offset:1] 
training_predictions = bayesian_lstm.predict(X_train)
training_df['log_energy_consumption'] = inverse_transform(training_predictions)
training_df['source'] = 'Training Prediction'

training_truth_df = pd.DataFrame()
training_truth_df['date'] = training_df['date']
training_truth_df['log_energy_consumption'] = resample_df['log_energy_consumption'].iloc[offset:n_train + offset:1] 
training_truth_df['source'] = 'True Values'

testing_df = pd.DataFrame()
testing_df['date'] = resample_df['date'].iloc[n_train + offset::1] 
testing_predictions = bayesian_lstm.predict(X_test)
testing_df['log_energy_consumption'] = inverse_transform(testing_predictions)
testing_df['source'] = 'Test Prediction'

testing_truth_df = pd.DataFrame()
testing_truth_df['date'] = testing_df['date']
testing_truth_df['log_energy_consumption'] = resample_df['log_energy_consumption'].iloc[n_train + offset::1] 
testing_truth_df['source'] = 'True Values'

evaluation = pd.concat([training_df, 
                        testing_df,
                        training_truth_df,
                        testing_truth_df
                        ], axis=0)

In [ ]:
fig = px.line(evaluation.loc[evaluation['date'].between('2016-04-14', '2016-04-23')],
                 x="date",
                 y="log_energy_consumption",
                 color="source",
                 title="Log of Appliance Energy Consumption in Wh vs Time")
fig.show()

# Uncertainty Quantification

The fact that stochastic dropouts are applied after each LSTM layer in the Bayesian LSTM enables users to interpret the model outputs as random samples from the posterior distribution of the target variable. 

This implies that by running multiple experiments/predictions, can approximate  parameters of the posterioir distribution, namely the mean and the variance, in order to create confidence intervals for each prediction.

In this example, we construct 99% confidence intervals that are three standard deviations away from the approximate mean of each prediction.

In [ ]:
n_experiments = 100

test_uncertainty_df = pd.DataFrame()
test_uncertainty_df['date'] = testing_df['date']

for i in range(n_experiments):
  experiment_predictions = bayesian_lstm.predict(X_test)
  test_uncertainty_df['log_energy_consumption_{}'.format(i)] = inverse_transform(experiment_predictions)

log_energy_consumption_df = test_uncertainty_df.filter(like='log_energy_consumption', axis=1)
test_uncertainty_df['log_energy_consumption_mean'] = log_energy_consumption_df.mean(axis=1)
test_uncertainty_df['log_energy_consumption_std'] = log_energy_consumption_df.std(axis=1)

test_uncertainty_df = test_uncertainty_df[['date', 'log_energy_consumption_mean', 'log_energy_consumption_std']]

In [ ]:
test_uncertainty_df['lower_bound'] = test_uncertainty_df['log_energy_consumption_mean'] - 3*test_uncertainty_df['log_energy_consumption_std']
test_uncertainty_df['upper_bound'] = test_uncertainty_df['log_energy_consumption_mean'] + 3*test_uncertainty_df['log_energy_consumption_std']

In [ ]:
import plotly.graph_objects as go

test_uncertainty_plot_df = test_uncertainty_df.copy(deep=True)
test_uncertainty_plot_df = test_uncertainty_plot_df.loc[test_uncertainty_plot_df['date'].between('2016-05-01', '2016-05-09')]
truth_uncertainty_plot_df = testing_truth_df.copy(deep=True)
truth_uncertainty_plot_df = truth_uncertainty_plot_df.loc[testing_truth_df['date'].between('2016-05-01', '2016-05-09')]

upper_trace = go.Scatter(
    x=test_uncertainty_plot_df['date'],
    y=test_uncertainty_plot_df['upper_bound'],
    mode='lines',
    fill=None,
    name='99% Upper Confidence Bound'
    )
lower_trace = go.Scatter(
    x=test_uncertainty_plot_df['date'],
    y=test_uncertainty_plot_df['lower_bound'],
    mode='lines',
    fill='tonexty',
    fillcolor='rgba(255, 211, 0, 0.1)',
    name='99% Lower Confidence Bound'
    )
real_trace = go.Scatter(
    x=truth_uncertainty_plot_df['date'],
    y=truth_uncertainty_plot_df['log_energy_consumption'],
    mode='lines',
    fill=None,
    name='Real Values'
    )

data = [upper_trace, lower_trace, real_trace]

fig = go.Figure(data=data)
fig.update_layout(title='Uncertainty Quantification for Energy Consumption Test Data',
                   xaxis_title='Time',
                   yaxis_title='log_energy_consumption (log Wh)')

fig.show()

#### Evaluating Uncertainty

Using multiple experiments above, 99% confidence intervals have been constructed for each the prediction of the target variable (the logarithm of appliance power consumption). While we can visually observe that the model is generally capturing the behavior of the time-series, approximately only 50% of the real data points lie within a 99% confidence interval from the mean prediction value.

Despite the relatively low percentage of points within the confidence interval, it must be noted that Bayesian Neural Networks only seek to quantify the epistemic model uncertainty and does not account for aleatoric uncertainty (i.e. noise).

In [ ]:
bounds_df = pd.DataFrame()

# Using 99% confidence bounds
bounds_df['lower_bound'] = test_uncertainty_plot_df['lower_bound']
bounds_df['prediction'] = test_uncertainty_plot_df['log_energy_consumption_mean']
bounds_df['real_value'] = truth_uncertainty_plot_df['log_energy_consumption']
bounds_df['upper_bound'] = test_uncertainty_plot_df['upper_bound']

bounds_df['contained'] = ((bounds_df['real_value'] >= bounds_df['lower_bound']) &
                          (bounds_df['real_value'] <= bounds_df['upper_bound']))

print("Proportion of points contained within 99% confidence interval:", 
      bounds_df['contained'].mean())

Proportion of points contained within 99% confidence interval: 0.6632124352331606


# Conclusions

- Bayesian LSTMs have been able to produce comparable performance to their frequentist counterparts (all else being equal)
- Stochastic dropout enables users to approximate the posterior distribution of the target variable, \
and thus construct confidence intervals for each prediction 
- Bayesian Neural Networks only attempt to account for epistemic model uncertainty and do not necessarily address aleatoric uncertainty
- Computational overhead for repeated/multiple Bayesian LSTM predictions at inference to construct confidence intervals represent a potential challenge for real-time inference use-cases.